In [1]:
# input_attention
import random

import tensorflow as tf
import tensorflow.keras.backend as K

from tensorflow.keras.layers import Dense, Permute, Softmax, Activation, Add

In [2]:
random.seed(42)

print(tf.__version__)
tf.random.set_seed(42)

2.11.0


- tf.keras.layers.Dense

- tf.keras.layers.Permute

In [3]:
ele01 = [random.random() for _ in range(4)]
ele02 = [ele01 for _ in range(5)]
ele03 = [ele02 for _ in range(7)]

X = tf.constant(ele03, dtype=tf.float32)

# print(X)
print(X.shape)
print()

X_tr = Permute((2, 1))(X)
# print(X_tr)
print(X_tr.shape)

Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB

(7, 5, 4)

(7, 4, 5)


2023-02-12 16:51:06.802421: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-02-12 16:51:06.802444: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [4]:
hidden_state = tf.constant([[random.random() for _ in range(3)] for _ in range(7)], dtype=tf.float32)
cell_state = tf.constant([[random.random() for _ in range(3)] for _ in range(7)], dtype=tf.float32)

print(tf.concat([hidden_state, cell_state], axis=-1).shape)

(7, 6)


In [5]:
n = X.shape[2]

concat_hs = K.repeat(tf.concat([hidden_state, cell_state], axis=-1), n)
print(concat_hs.shape)

(7, 4, 6)


In [6]:
T = 6
hs = Dense(T)(concat_hs)
print(Dense(T)(hs).shape)

(7, 4, 6)


In [7]:
ux = Dense(T)(Permute((2, 1))(X))
print(ux.shape)

(7, 4, 6)


In [8]:
print(hs[0, 0, :])

print(ux[0, 0, :])

tf.Tensor([ 0.48454535  0.62506074  0.4443849   0.8170673  -0.84375733  0.5550963 ], shape=(6,), dtype=float32)
tf.Tensor([ 0.8329017  -0.8097319  -0.5543615  -0.3726091  -0.51796836 -0.48469996], shape=(6,), dtype=float32)


### tf.keras.layers.Add

- tf.math.tanh

- tf.keras.layers.Activation

In [9]:
temp_add = Add()([hs, ux])

print(temp_add.shape)

print(temp_add[0, 0, :])

(7, 4, 6)
tf.Tensor([ 1.3174471  -0.18467116 -0.10997662  0.44445822 -1.3617257   0.07039636], shape=(6,), dtype=float32)


In [10]:
tanh_math_add = tf.math.tanh(temp_add)
print(tanh_math_add.shape)

(7, 4, 6)


In [11]:
tanh_act_add = Activation(activation='tanh')(temp_add)
print(tanh_act_add.shape)

(7, 4, 6)


In [12]:
diff_tanh_add = tanh_math_add - tanh_act_add
print(sum(sum(sum(diff_tanh_add))))

tf.Tensor(0.0, shape=(), dtype=float32)


In [13]:
e_add = Dense(1)(tanh_act_add)
print(e_add[:, :, 0])
print(e_add.shape)

tf.Tensor(
[[-0.38876498 -0.33333313 -0.41459054 -0.40624875]
 [ 0.16953865  0.46228567  0.33247373  0.35806623]
 [ 0.10799703  0.36778215  0.25185204  0.27430263]
 [ 0.10391789  0.11568136  0.1148102   0.11408954]
 [ 0.21432039  0.19410983  0.18942888  0.18702151]
 [-0.09489743 -0.26364508 -0.2191125  -0.2337653 ]
 [-0.26094747 -0.2114221  -0.27438432 -0.26838902]], shape=(7, 4), dtype=float32)
(7, 4, 1)


In [14]:
attn_add = Softmax()(e_add)
print(attn_add[:, :, 0])
print(attn_add.shape)

tf.Tensor(
[[1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]], shape=(7, 4), dtype=float32)
(7, 4, 1)


In [15]:
attn_add = Softmax()(Permute((2, 1))(e_add))
print(attn_add[:, 0, :])
print(attn_add.shape)

tf.Tensor(
[[0.2491175  0.26331645 0.24276626 0.24479984]
 [0.21166107 0.2836478  0.24911667 0.2555745 ]
 [0.21587539 0.27991515 0.24927492 0.25493458]
 [0.24795388 0.2508879  0.25066942 0.25048885]
 [0.2545515  0.2494585  0.24829352 0.2476965 ]
 [0.27791253 0.23475882 0.24544951 0.24187921]
 [0.24813825 0.26073682 0.24482638 0.2462986 ]], shape=(7, 4), dtype=float32)
(7, 1, 4)


### tf.concat

- tf.math.tanh

- tf.keras.layers.Activation

In [16]:
temp_concat = tf.concat([hs, ux], axis=-1)

print(temp_concat.shape)

print(temp_concat[0, 0, :])

(7, 4, 12)
tf.Tensor(
[ 0.48454535  0.62506074  0.4443849   0.8170673  -0.84375733  0.5550963
  0.8329017  -0.8097319  -0.5543615  -0.3726091  -0.51796836 -0.48469996], shape=(12,), dtype=float32)


In [17]:
tanh_math_concat = tf.math.tanh(temp_concat)
print(tanh_math_concat.shape)

(7, 4, 12)


In [18]:
tanh_act_concat = Activation(activation='tanh')(temp_concat)
print(tanh_act_concat.shape)

(7, 4, 12)


In [19]:
diff_tanh = tanh_math_concat - tanh_act_concat
print(sum(sum(sum(diff_tanh))))

tf.Tensor(0.0, shape=(), dtype=float32)


In [20]:
e_act = Dense(1)(tanh_act_concat)
print(e_act[:, :, 0])
print(e_act.shape)

tf.Tensor(
[[1.4946017  0.6969331  1.0590069  0.98671514]
 [2.0720077  1.2743387  1.6364125  1.5641207 ]
 [2.069405   1.2717364  1.63381    1.5615184 ]
 [1.6820798  0.8844111  1.2464849  1.1741931 ]
 [1.8753824  1.0777137  1.4397874  1.3674957 ]
 [1.600548   0.8028792  1.164953   1.0926613 ]
 [1.455032   0.6573632  1.019437   0.9471452 ]], shape=(7, 4), dtype=float32)
(7, 4, 1)


In [21]:
attn_input_act = Softmax()(e_act)
print(attn_input_act[:, :, 0])
print(attn_input_act.shape)

tf.Tensor(
[[1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]], shape=(7, 4), dtype=float32)
(7, 4, 1)


In [22]:
attn_input_act = Softmax()(Permute((2, 1))(e_act))
print(attn_input_act[:, 0, :])
print(attn_input_act.shape)

tf.Tensor(
[[0.37050438 0.16686691 0.2396718  0.22295694]
 [0.37050447 0.16686691 0.23967174 0.2229569 ]
 [0.3705044  0.16686693 0.23967175 0.22295693]
 [0.3705044  0.16686693 0.23967177 0.22295693]
 [0.3705044  0.16686693 0.23967175 0.22295691]
 [0.3705044  0.1668669  0.23967175 0.22295691]
 [0.3705044  0.1668669  0.23967175 0.22295691]], shape=(7, 4), dtype=float32)
(7, 1, 4)
